##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# O modelo Sequencial

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/keras/sequential_model"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/guide/keras/sequential_model.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/guide/keras/sequential_model.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/guide/keras/sequential_model.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Configuração

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Quando usar um modelo sequencial

Um modelo `Sequential` é apropriado para **uma pilha simples de camadas** em que cada camada tem **exatamente um tensor de entrada e um tensor de saída**.

Esquematicamente, o seguinte modelo `Sequential`:

In [ ]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)
# Call model on a test input
x = tf.ones((3, 3))
y = model(x)

é equivalente a esta função:

In [ ]:
# Create 3 layers
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

# Call layers on a test input
x = tf.ones((3, 3))
y = layer3(layer2(layer1(x)))

Um modelo sequencial **não é apropriado** quando:

- Seu modelo tem múltiplas entradas ou saídas
- Qualquer uma de suas camadas tem múltiplas entradas ou saídas
- Você precisar fazer compartilhamento de camadas
- Você quiser uma topologia não linear (por exemplo, uma conexão residual, um modelo multi-branch)

## Criando um modelo sequencial

Você pode criar um modelo sequencial passando uma lista de camadas para o construtor Sequential:

In [ ]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)

Suas camadas são acessíveis através do atributo `layers`:

In [ ]:
model.layers

Você também pode criar um modelo sequencial de forma incremental através do método `add()`:

In [ ]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu"))
model.add(layers.Dense(3, activation="relu"))
model.add(layers.Dense(4))

Observe que há também um método `pop()` correspondente para remover camadas: um modelo sequencial se comporta de maneira muito semelhante a uma lista de camadas.

In [ ]:
model.pop()
print(len(model.layers))  # 2

Observe também que o construtor Sequential aceita um argumento `name`, assim como qualquer camada ou modelo em Keras. Isto é útil para anotar grafos do TensorBoard com nomes semanticamente significativos.

In [ ]:
model = keras.Sequential(name="my_sequential")
model.add(layers.Dense(2, activation="relu", name="layer1"))
model.add(layers.Dense(3, activation="relu", name="layer2"))
model.add(layers.Dense(4, name="layer3"))

## Especificando o formato de entrada com antecedência

Geralmente, todas as camadas em Keras precisam conhecer o formato de suas entradas para poderem criar seus pesos. Então, quando você cria uma camada como esta, inicialmente ela não tem pesos:

In [ ]:
layer = layers.Dense(3)
layer.weights  # Empty

Ela cria seus pesos na primeira vez que é chamada numa entrada, pois a forma dos pesos depende do formato das entradas:

In [ ]:
# Call layer on a test input
x = tf.ones((1, 4))
y = layer(x)
layer.weights  # Now it has weights, of shape (4, 3) and (3,)

Naturalmente, isto também vale para os modelos sequenciais. Quando você instancia um modelo sequencial sem um formato de entrada, ele não é "construído": ele não tem pesos (e chamar `model.weights` resulta num erro que informa exatamente isso). Os pesos são criados quando o modelo vê pela primeira vez alguns dados de entrada:

In [ ]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)  # No weights at this stage!

# At this point, you can't do this:
# model.weights

# You also can't do this:
# model.summary()

# Call the model on a test input
x = tf.ones((1, 4))
y = model(x)
print("Number of weights after calling the model:", len(model.weights))  # 6

Depois que um modelo é "construído", você pode chamar seu método `summary()` para exibir seu conteúdo:

In [ ]:
model.summary()

No entanto, pode ser muito útil ao construir um modelo sequencial de forma incremental, poder mostrar o resumo do modelo até o momento, incluindo o formato de saída atual. Nesse caso, você deve iniciar seu modelo passando um objeto `Input` para seu modelo, para que ele conheça seu formato de entrada desde o início:

In [ ]:
model = keras.Sequential()
model.add(keras.Input(shape=(4,)))
model.add(layers.Dense(2, activation="relu"))

model.summary()

Observe que o objeto `Input` não é exibido como parte de `model.layers`, já que não é uma camada:

In [ ]:
model.layers

Uma alternativa simples é simplesmente passar um argumento `input_shape` para sua primeira camada:

In [ ]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu", input_shape=(4,)))

model.summary()

Modelos construídos com um formato de entrada predefinido como este sempre têm pesos (mesmo antes de ver quaisquer dados) e sempre têm um formato de saída definido.

Em geral, é uma prática recomendada sempre especificar antecipadamente o formato de entrada de um modelo sequencial se você souber qual é.

## Um workflow de depuração comum: `add()` + `summary()`

Ao construir uma nova arquitetura sequencial, é útil empilhar camadas de forma incremental com `add()` e imprimir resumos de modelos com frequência. Por exemplo, isso permite que você monitore como uma pilha de camadas `Conv2D` e `MaxPooling2D` está diminuindo a resolução (downsampling) dos mapas de características da imagem:

In [ ]:
model = keras.Sequential()
model.add(keras.Input(shape=(250, 250, 3)))  # 250x250 RGB images
model.add(layers.Conv2D(32, 5, strides=2, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))

# Can you guess what the current output shape is at this point? Probably not.
# Let's just print it:
model.summary()

# The answer was: (40, 40, 32), so we can keep downsampling...

model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))

# And now?
model.summary()

# Now that we have 4x4 feature maps, time to apply global max pooling.
model.add(layers.GlobalMaxPooling2D())

# Finally, we add a classification layer.
model.add(layers.Dense(10))

Muito prático, não é mesmo?


## O que fazer depois de que você tiver um modelo

Assim que a arquitetura do seu modelo estiver pronta, você vai querer:

- Treinar seu modelo, avaliá-lo e executar a inferência. Consulte nosso [guia de treinamento e avaliação com os loops integrados](https://www.tensorflow.org/guide/keras/train_and_evaluate/)
- Salvar seu modelo no disco e restaurá-lo. Veja nosso [guia para serialização e salvamento](https://www.tensorflow.org/guide/keras/save_and_serialize/).
- Acelerar o treinamento do modelo aproveitando várias GPUs. Consulte nosso [guia para treinamento multi-GPU e distribuído](https://keras.io/guides/distributed_training/).

## Extração de características com um modelo sequencial

Depois que um modelo sequencial for construído, ele se comporta como um [modelo de API funcional](https://www.tensorflow.org/guide/keras/functional/). Isto significa que cada camada tem um atributo `input` e `output`. Esses atributos podem ser usados ​​para fazer coisas legais, como criar rapidamente um modelo que extrai as saídas de todas as camadas intermediárias em um modelo sequencial:

In [ ]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=[layer.output for layer in initial_model.layers],
)

# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

Aqui está um exemplo semelhante que extrai apenas características (features) de uma camada:

In [ ]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu", name="my_intermediate_layer"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=initial_model.get_layer(name="my_intermediate_layer").output,
)
# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

## Aprendizado por transferência com um modelo sequencial

O aprendizado por transferência (transfer learning) consiste em congelar as camadas inferiores em um modelo e treinar apenas as camadas superiores. Se você não estiver familiarizado com ele, não deixe de ler nosso [guia sobre aprendizado por transferência](https://www.tensorflow.org/guide/keras/transfer_learning/).

Aqui estão dois esquemas comuns de aprendizado por transferência envolvendo modelos sequenciais.

Primeiro, digamos que você tenha um modelo sequencial e queira congelar todas as camadas, exceto a última. Nesse caso, você pode simplesmente iterar sobre `model.layers` e definir `layer.trainable = False` em cada camada, exceto na última. Assim:

```python
model = keras.Sequential([
    keras.Input(shape=(784)),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10),
])

# Presumably you would want to first load pre-trained weights.
model.load_weights(...)

# Freeze all layers except the last one.
for layer in model.layers[:-1]:
  layer.trainable = False

# Recompile and train (this will only update the weights of the last layer).
model.compile(...)
model.fit(...)
```

Outra estratégia comum é usar um modelo sequencial para empilhar um modelo pré-treinado e algumas camadas de classificação recém-inicializadas. Assim:

```python
# Load a convolutional base with pre-trained weights
base_model = keras.applications.Xception(
    weights='imagenet',
    include_top=False,
    pooling='avg')

# Freeze the base model
base_model.trainable = False

# Use a Sequential model to add a trainable classifier on top
model = keras.Sequential([
    base_model,
    layers.Dense(1000),
])

# Compile &amp; train
model.compile(...)
model.fit(...)
```

Se você usa aprendizado por transferência, provavelmente vai utilizar esses dois padrões com frequência.

Isso é tudo o que você precisa saber sobre modelos sequenciais!

Para saber mais sobre a construção de modelos em Keras, consulte:

- [Guia para a API Functional](https://www.tensorflow.org/guide/keras/functional/)
- [Guia para criar novas camadas e modelos usando subclasses](https://www.tensorflow.org/guide/keras/custom_layers_and_models/)